In [481]:
import requests as r
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [482]:
http_header = {
    'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
    'x-requested-with': 'XMLHttpRequest'
}

In [483]:
session = r.Session()
session.headers.update(http_header)

In [484]:
dfB = pd.read_excel('list_of_address.xlsx', usecols='B', names=['시작주소'], encoding='utf-8')
dfC = pd.read_excel('list_of_address.xlsx', usecols='C', names=['도로명주소'], encoding='utf-8')

In [485]:
dfB.head(5)

,시작주소
0,서울특별시 종로구 성균관로 25-2
1,서울특별시 관악구 난향9길 15-3
2,남부순환로 2406
3,서울특별시 관악구 신림로36길 16
4,서울특별시 관악구 난곡로 63


In [486]:
dfC.head(10)

,도로명주소
0,서울특별시 관악구 난향9길 15-3
1,남부순환로 2406
2,서울특별시 관악구 신림로36길 16
3,서울특별시 관악구 난곡로 63
4,서울특별시 관악구 남부순환로172길 16-7
5,서울특별시 관악구 남부순환로 1391
6,서울특별시 관악구 쑥고개로2가길 22
7,서울특별시 관악구 신림로 248
8,서울특별시 관악구 신림로17길 32
9,서울특별시 관악구 봉천로12길 65


In [487]:
B_array = np.array(dfB.values)
C_array = np.array(dfC.values)
B_list = list(B_array)
print(B_list)
C_list = list(C_array)
print(C_list)

In [488]:
start = B_list[1]
start = (str(start[0]))
end = C_list[1] 
end = (str(end[0]))

print(start)
print(end)

In [489]:
search_distance_url_base = 'https://m.map.naver.com/spirra/findCarRoute.nhn?route=route3&output=json&result=web3&coord_type=latlng&search=2&car=0&mileage=12.4'

In [490]:
def SEARCH_DISTANCE_URL(start_point, end_point):
    return search_distance_url_base+'&start={}&destination={}'.format(start_point, end_point)

In [491]:
def SEARCH_POINT_URL(q):
    return 'https://m.map.naver.com/apis/search/poi?query={}&page=1'.format(q)

In [492]:
def GET_END_POINT(q):
    res = session.get(SEARCH_POINT_URL(q)).text
    res_dict = json.loads(res)
    x = res_dict['result']['address']['list'][0]['x']
    y = res_dict['result']['address']['list'][0]['y']
    name = res_dict['result']['address']['list'][0]['name']
    return '{},{},{}'.format(x, y, name)

In [493]:
def GET_INFO(start, end):
    start_point = GET_END_POINT(start)
    end_point = GET_END_POINT(end)
    
    res = session.get(SEARCH_DISTANCE_URL(start_point, end_point)).text
    res_dict = json.loads(res)
    
    target = res_dict['routes'][0]['summary']
    start = target['start']
    s_address = start['address']
    s_location = start['location']
    end = target['end']
    e_address = end['address']
    e_location = end['location']
    distance = target['distance']
    sec = target['duration']
    taxi_fare = target['taxi_fare']

    x1=float(s_location[:11])
    y1=float(s_location[12:])
    x2=float(e_location[:11])
    y2=float(e_location[12:])
    
    
    print('검색완료')
    print('출발지: {}'.format(s_address))
    print('출발지 위경도: {}'.format(s_location))
    print('도착지: {}'.format(e_address))
    print('도착지 위경도: {}'.format(e_location))
    print('추천경로 이동 거리: {}km'.format(distance/1000))
    print('예상 소요시간: {}분'.format(sec/60))
    print('예상 택시요금: {}원'.format(taxi_fare))

    print('출발지 경도: {}'.format(x1))
    print('출발지 경도: {}'.format(y1))
     
    s_loc =  [float(s_location[12:]), float(s_location[:11])]
    e_loc =  [float(e_location[12:]), float(e_location[:11])]
    import folium
    map_osm = folium.Map(location=[float(s_location[12:]), float(s_location[:11])], zoom_start=12)
    map_osm = folium.Map(location=[float(e_location[12:]), float(e_location[:11])], zoom_start=12) 
    folium.Marker(s_loc, popup='출발지', icon=folium.Icon(color='blue',icon='info-sign')).add_to(map_osm)
    folium.Marker(e_loc, popup='도착지', icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)        
   
    map_osm.save('C:/Users/sjsje/mywork/map4.html') #파일이 저장될 위치
    
    

In [494]:
GET_INFO(start, end)